In [ ]:
from collections import defaultdict

import requests
from tqdm import tqdm

In [ ]:
resp = requests.get("https://gist.githubusercontent.com/dracos/dd0668f281e685bad51479e5acaadb93/raw/6bfa15d263d6d5b63840a8e5b64e04b382fdb079/valid-wordle-words.txt")

In [ ]:
words = set(w for w in resp.text.split("\n") if len(w)==5)

In [ ]:
def sparse(word, guess):
    enum = {i:l for (i,l) in enumerate(guess) if word[i]==l}
    others = "".join(l for l in set(guess) if l in word and l not in enum.values())
    if len(others)>0:
        enum[-1] = others
    return enum

In [ ]:
def ori_tuple(word, guess):
    rtuple = list(0 for _ in range(len(guess)))
    for i,l in enumerate(guess):
        if l in word:
            rtuple[i] = 1
        if word[i]==l:
            rtuple[i] = 2
    return tuple(rtuple)

In [ ]:
def gen_wordle(words, guesses):
    rset = words.copy()
    for guess, guess_tuple in guesses.items():
        for i,xl in enumerate(guess_tuple):
            removewords = set()
            for word in rset:
                if xl==0 and guess[i] in word:
                    removewords.add(word)
                elif xl==1 and guess[i] not in word:
                    removewords.add(word)
                elif xl==2 and guess[i]!=word[i]:
                    removewords.add(word)
            rset -= removewords
    return rset

In [ ]:
def next_wordle(words, no_bar=False):
    rdict = defaultdict(lambda: defaultdict(int))
    for word in tqdm(words, disable=no_bar):
        for guess in words:
            rdict[guess][ori_tuple(word, guess)] += 1
    ml = max(len(v) for v in rdict.values())
    score = {w:(ml-len(v))*len(words)//2 for w,v in rdict.items()}
    for w,v in rdict.items():
        for t,n in v.items():
            score[w] += abs(n-len(words)//2)
    min_score = score[min(score, key=score.get)]
    ppp = set()
    for word in sorted(score, key=score.get):
        if score[word]==min_score:
            ppp.add(word)
        else:
            break
    return ppp

In [ ]:
next_wordle(gen_wordle(words, {"tares": (0,0,0,0,0)}), no_bar=True)